In [ ]:
from datetime import datetime, timedelta
from chat_history import insert_data, get_last_conversation, get_prev_queries
from vector_retrieval import create_context
import mysql.connector
import weaviate
import tiktoken
import requests
import openai
import json


openai.api_key = '<<Your API Key>>'
auth_config = weaviate.auth.AuthApiKey(api_key="<<Your API Key>>")
client = weaviate.Client(
    url = "Replace with your endpoint",  
    auth_client_secret=auth_config,
    additional_headers = {
        "X-Cohere-Api-Key" : "<<Your API Key>>" # Replace with your API key
    }
)

In [ ]:
import cohere
openai.api_key = '<<Your API Key>>'
co = cohere.Client('<<Your API Key>>')

In [ ]:
def generate_response(
    prompt,
    debug=False,
    max_tokens=100,
    stop_sequence=None,
):
    """
    Answer a question based on the most similar context from the dataframe texts
    """
    try:
        #print(instruction.format(context, question))
        response = openai.Completion.create(
            model = "text-davinci-003",
            prompt=prompt,
            temperature=0.5,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=True,
            best_of = 
        )

        generated_text = ""
        for line in response:
            generated_text += line.choices[0].text  # extract the text from the StreamingText object
        
        return generated_text.split("###")[0].strip()

    except Exception as e:
        print(e)
        return ""

In [ ]:

def calculate_tokens(prompt):
    encoding = tiktoken.encoding_for_model("text-davinci-003")
    num_tokens = len(encoding.encode(prompt))
    return num_tokens

In [ ]:
def chatbot(uuid, vec_class, question, debug=False):
    # Initialize Database
    mydb = mysql.connector.connect(
        host="<<Database endpoint>>",
        user="",
        password="",
        database=""
    )

    # Retrieve last conversation for the vec_class
    last_conversation = get_last_conversation(mydb, uuid, vec_class)
    prev_queries = get_prev_queries(mydb, uuid, vec_class)

    # Get Context for answering question
    context = create_context(
        question,
        prev_queries,
        vec_class  
    )

    # Print context while debugging

        
    
    # If there was a previous conversation, include it in the prompt
    if last_conversation is not None:
        prompt = f'''You are an AI language model that provides answers based on the given context and previous conversations. Your task is to answer the following question using the provided Knowledge Base and previous conversations all delimited by three backticks:
            If any relevant link is available in the Knowledge Base make sure to include them in the answer.
            Do not attempt to make up any answer yourself.
            \n\n 
            ```Knowledge Base: \n{context}```\n\n---\n\n
            ```Previous Conversations: {last_conversation}```\n
            Question: {question}\n
            Answer:
        '''
    else:
        prompt = f'''You are an AI language model that provides answers based on the given context. Your task is to answer the following question using the provided Knowledge Base delimited by three backticks: 
            If no question is asked and the user simply greets you. You greet them back politely.
            If relevant link is available in the Knowledge Base include them in the answer.
            Do not attempt to make up any answer yourself.
            \n\n ```Knowledge Base: \n{context}```\n\n---\n\n
            Question: {question}\n
            Answer:
        '''
    
    num_tokens = calculate_tokens(prompt)

    if debug:
        print("Last 3 Queries: ", prev_queries)
        print("Last 3 Conversation: ", last_conversation)
        print(context)
        print("Num Tokens: ", num_tokens)
    
    # Generate response from OpenAI API
    response = generate_response(prompt)
    
    # Insert conversation into the database
    # uuid = str(uuid.uuid4())
    # chat_conv = f"{context}\n\n---\n\nQuestion: {question}\nAnswer: {response}"
    chat_conv = f"Question: {question} Answer: {response}"
    insert_data(mydb, vec_class, uuid, question, chat_conv)

    mydb.close()
    
    # Return response to the user
    return response

In [ ]:
query =''
uuid=''
vec_class = ''
lang_code = detect_lang(query)
print(lang_code)
if lang_code == 'EN':
    result = chatbot(uuid, vec_class, query, debug = True)
else:
    trans_query = user_lang(query, 'EN')
    answer = chatbot(uuid, vec_class, trans_query, debug = True)
    result = user_lang(answer, lang_code)

print(result)

